### 載入套件

In [1]:
import os
import sys
import cv2
import pafy
import json
from time import time

dir_path = os.getcwd()
sys.path.append(dir_path + r'\build\python\openpose\Release')
os.environ['PATH'] = os.environ['PATH'] + ';' + dir_path + r'\build\x64\Release;' + dir_path + r'\build\bin;'
import pyopenpose as op

### 路徑、參數設定及初始化

In [2]:
img_path = r'.\media\images'
vid_path = r'.\media\videos'
json_file = r'..\Unity\OpenPose\Assets\Resources\keypoints.json'

params = dict()
params['model_folder'] = r'.\models\\'
params['net_resolution'] = '240x240' # 16的倍數

opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()
datum = op.Datum()

### 函式

In [3]:
def writeJson(opData):
    kpData = []
    pplData = []
    
    try:
        for i in range(opData.shape[0]):
            for j in range(25):
                for k in range(3):
                    kpData.append(float(opData[0, j, k]))        
            pplData.append({'person_id': [i], 'pose_keypoints': kpData})        
        jsonDict = {'people': pplData}
    
        with open(json_file, 'w', encoding = 'UTF-8') as file:
            json.dump(jsonDict, file)
            
    except (AttributeError, PermissionError):
        pass

### 影片

In [31]:
vidFile = 'JCF0NCNynrc.mp4'
video = os.path.join(vid_path, vidFile)
vid = cv2.VideoCapture(video)

while True:
    t_start = time()
    ret, frame = vid.read()
    if not ret:
        break
    frame = cv2.resize(frame, (1024, 576))
    
    datum.cvInputData = frame    
    opWrapper.emplaceAndPop(op.VectorDatum([datum]))
    cvOutput = datum.cvOutputData
    
    outputData = datum.poseKeypoints
    writeJson(outputData)
    
    FPS = 1 / (time() - t_start)
    
    cv2.putText(cvOutput, 'FPS: %.1f' % FPS, (10, 35), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(cvOutput, 'FPS: %.1f' % FPS, (10, 35), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(cvOutput, 'Press "Esc" to stop.', (10, cvOutput.shape[0] - 11), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(cvOutput, 'Press "Esc" to stop.', (10, cvOutput.shape[0] - 11), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)
    
    cv2.imshow('OpenPose - Python API', cvOutput)
    if cv2.waitKey(1) == 27: # 按Esc關閉
        break
        
cv2.destroyAllWindows()

### 下載影片

In [15]:
url = 'https://www.youtube.com/watch?v=JCF0NCNynrc' # video url
# ----------------------------------------------------------------------------------------------------- #
video = pafy.new(url)
fileName = video.videoid
for vst in video.videostreams:
    if str(vst) == 'video:mp4@1280x720':
        video720p = vst # get resolution 1280x720 mp4
        break
# ----------------------------------------------------------------------------------------------------- #
dl_path = os.path.join(vid_path, fileName + '.mp4')
video720p.download(filepath = dl_path)

  5,873,123.0 Bytes [100.00%] received. Rate: [22584 KB/s].  ETA: [0 secs]    


### 圖片

In [4]:
#imgFile = ['COCO_val2014_000000000241.jpg', 'COCO_val2014_000000000459.jpg']
imgFile = ['8404579123_a6815b6e1a_z.jpg']

for image in imgFile:  
    img = os.path.join(img_path, image)
    datum.cvInputData = cv2.imread(img)
    opWrapper.emplaceAndPop(op.VectorDatum([datum]))
    
    outputData = datum.poseKeypoints
    writeJson(outputData)
    
    cv2.imshow('OpenPose - Python API', datum.cvOutputData)    
    cv2.waitKey(0)
    
cv2.destroyAllWindows()